# 1. Creación de un Prompt


## Objetivo del Prompt:



En mi tiempo libre me dedico a coleccionar cartas de Pokémon TCG, para lo cual estoy constantemente buscando y comparando precios de las cartas en diferentes plataformas como TCGplayer o PriceCharting. Además, también pierdo mucho tiempo buscando la probabilidad de que salga cada carta en los sobres, así como información acerca de la colección a la que pertenece.

Por ello, mi intención es crear un prompt cuya entrada sea el nombre del pokemon, numeración y colección a la que pertenece, y cuyo output sea información relevante de la colección a la que pertenece, precios de venta de dicha carta en función de su grade y su pullrate.

## Redacción del prompt:


Actúa como un experto en coleccionismo de Pokémon TCG. Busca el valor de mercado en dólares de la carta solicitada y la probabilidad de pullrate de que salga en un sobre. Dame una respuesta que cumpla el siguiente formato:
1. Una breve información sobre la expansión a la que pertenece la carta: nombre, fecha de lanzamiento, impresión aproximada, número de cartas que componen la colección, etc.
2. Una tabla donde aparezcan el precio y el pullrate.

Un ejemplo de respuesta para el Charizard ex 199/165 de la colección 151 sería el siguiente:

La carta Charizard ex 199/165 pertenece a la expansión Scarlet & Violet—151 del Juego de Cartas Coleccionables Pokémon. A continuación, se proporciona información relevante sobre esta expansión:

1. Información sobre la expansión:

Nombre: Scarlet & Violet—151
Fecha de lanzamiento: 22 de septiembre de 2023
Número de cartas en la colección: 165 cartas regulares más cartas secretas, totalizando más de 200 cartas
Características destacadas: Esta expansión celebra los 151 Pokémon originales de la región de Kanto, presentando ilustraciones especiales y cartas raras de Pokémon emblemáticos.

2. Tabla de precios y pullrate del Charizard ex 199/165:

Estado de la carta | Precio estimado (USD) | Probabilidad de aparición en un sobre
Sin gradear	| $209.99	| 0.44%

# 2. Solución al problema


In [1]:
pip install langchain langchain-openai

# 2.1. Modelo A

In [2]:
openai_api_key=""

In [3]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain import PromptTemplate

# Configuramos el modelo LLM
llm = ChatOpenAI(temperature=0.0, model="gpt-4o", api_key = openai_api_key)

In [4]:
plantilla = """Actúa como un experto en coleccionismo de Pokémon TCG. Busca el valor de mercado en dólares del {Carta_Pokémon} y la probabilidad de pullrate de que salga en un sobre. Dame una respuesta que cumpla el siguiente formato:
1. Una breve información sobre la expansión a la que pertenece la carta: nombre, fecha de lanzamiento, impresión aproximada, número de cartas que componen la colección, etc.
2. Una tabla donde aparezcan el precio y el pullrate.

Un ejemplo de respuesta para el "Pikachu ex 238/191" de la colección "Surging Sparks" sería el siguiente:

La carta Pikachu ex 238/191 pertenece a la expansión Scarlet & Violet—Surging Sparks del Juego de Cartas Coleccionables Pokémon. A continuación, se proporciona información relevante sobre esta expansión:

1. Información sobre la expansión:

Nombre: Scarlet & Violet—Surging Sparks
Fecha de lanzamiento: 8 de noviembre de 2024
Número de cartas en la colección: 191 cartas regulares más cartas secretas, totalizando más de 200 cartas
Características destacadas: Esta expansión introduce nuevas mecánicas de juego y presenta ilustraciones especiales de Pokémon populares, incluyendo cartas raras y de arte especial.

2. Tabla de precios y pullrate del Pikachu ex 238/191:

Estado de la carta |	Precio estimado (USD) |	Probabilidad de aparición en un sobre
Sin gradear |	$458.54 |	0.10% """

# Creamos la plantilla
prompt_template = PromptTemplate.from_template(plantilla)

# Formateamos el prompt
Carta_Pokémon = "Greninja 214/167 de Twilight Masquerade"
prompt = prompt_template.format(Carta_Pokémon = Carta_Pokémon)

# Generamos la respuesta
respuesta = llm(prompt)
print (respuesta)

<ipython-input-4-e679b76c12cc>:29: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = llm(prompt)


content='La carta Greninja 214/167 pertenece a la expansión Twilight Masquerade del Juego de Cartas Coleccionables Pokémon. A continuación, se proporciona información relevante sobre esta expansión:\n\n1. Información sobre la expansión:\n\nNombre: Twilight Masquerade  \nFecha de lanzamiento: 15 de marzo de 2023  \nNúmero de cartas en la colección: 167 cartas regulares más cartas secretas, totalizando más de 200 cartas  \nCaracterísticas destacadas: Esta expansión se centra en Pokémon de tipo Fantasma y Siniestro, con ilustraciones temáticas de eventos nocturnos y festivales. Incluye cartas raras y de arte especial que son muy buscadas por los coleccionistas.\n\n2. Tabla de precios y pullrate del Greninja 214/167:\n\n| Estado de la carta | Precio estimado (USD) | Probabilidad de aparición en un sobre |\n|--------------------|-----------------------|--------------------------------------|\n| Sin gradear        | $350.00               | 0.05%                                |\n\nEsta infor

# 2.2. Modelo B

En este caso he querido solicitar al modelo un listado de las 5 cartas más buscadas de cada set, para luego utilizar el resultado como input y de nuevo solicitar una tabla con los precios y pullrate de cada una de ellas.

## Redacción del primer prompt:

Actúa como un experto en coleccionismo de Pokémon TCG. Investiga las cartas más buscadas por los coleccionistas de Pokémon TCG de la expansión solicitada. Como respuesta dame únicamente los 5 nombres de las cartas con su numeración.

Un ejemplo de respuesta para la expansión de 151 sería el siguiente:

"Charizard ex 199/165, Blastoise ex 200/165, Venusaur ex 198/165, Zapdos ex 202/165, Charmander 168/165"

In [5]:
# Creamos la primera plantilla
primera_plantilla = """Actúa como un experto en coleccionismo de Pokémon TCG. Investiga las cartas más buscadas por los coleccionistas de Pokémon TCG de la expansión de {Expansión_Pokémon}. Como respuesta dame únicamente los 5 nombres de las cartas con su numeración.

Un ejemplo de respuesta para la expansión de 151 sería el siguiente:

"Charizard ex 199/165, Blastoise ex 200/165, Venusaur ex 198/165, Zapdos ex 202/165, Charmander 168/165" """

primer_prompt = PromptTemplate(
    input_variables=["Expansión_Pokémon"],
    template=primera_plantilla
)

# Creamos la primera cadena
cadena_uno = LLMChain(llm=llm, prompt=primer_prompt)

<ipython-input-5-2882fc84ce18>:14: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  cadena_uno = LLMChain(llm=llm, prompt=primer_prompt)


## Redacción del segundo prompt:

Actúa como un experto en coleccionismo de Pokémon TCG. Busca el valor de mercado en dólares y la probabilidad de pullrate de que salga en un sobre las 5 cartas seleccionadas. Dame una respuesta que cumpla el siguiente formato:
1. Una breve información sobre la expansión a la que pertenecen las cartas: nombre, fecha de lanzamiento, impresión aproximada, número de cartas que componen la colección, etc.
2. Una tabla donde aparezcan los cinco Pokémon con el precio y el pullrate.

Un ejemplo de respuesta para Charizard ex 199/165, Blastoise ex 200/165, Venusaur ex 198/165, Zapdos ex 202/165, Charmander 168/165 de la expansión 151 sería el siguiente:

La expansión Escarlata y Púrpura-151 del Juego de Cartas Coleccionables Pokémon se lanzó el 22 de septiembre de 2023. Esta colección especial incluye a los 151 Pokémon originales de la región de Kanto, actualizados con las mecánicas de la serie Escarlata y Púrpura.

A continuación, se presenta una tabla con las cinco cartas más destacadas de esta expansión, junto con su valor de mercado estimado en dólares estadounidenses y la probabilidad aproximada de obtenerlas en un sobre (pull rate):

Carta	Número | Valor de Mercado (USD) | Pull Rate Estimado
Charizard ex	199/165 |	$121.47 |	1 en 94 sobres
Blastoise ex	200/165	| $77.25 |	1 en 71 sobres
Venusaur ex	198/165	| $76.69 |	1 en 283 sobres
Zapdos ex	202/165	| $55.31 |	Datos no disponibles
Charmander	168/165	| $50.98 |	Datos no disponibles

In [6]:
# Creamos la segunda plantilla
segunda_plantilla= """ Actúa como un experto en coleccionismo de Pokémon TCG. Busca el valor de mercado en dólares y la probabilidad de pullrate de que salga en un sobre las 5 cartas: {Cartas_Pokémon}. Dame una respuesta que cumpla el siguiente formato:
1. Una breve información sobre la expansión a la que pertenecen las cartas: nombre, fecha de lanzamiento, impresión aproximada, número de cartas que componen la colección, etc.
2. Una tabla donde aparezcan los cinco Pokémon con el precio y el pullrate.

Un ejemplo de respuesta para Charizard ex 199/165, Blastoise ex 200/165, Venusaur ex 198/165, Zapdos ex 202/165, Charmander 168/165 de la expansión 151 sería el siguiente:

La expansión Escarlata y Púrpura-151 del Juego de Cartas Coleccionables Pokémon se lanzó el 22 de septiembre de 2023. Esta colección especial incluye a los 151 Pokémon originales de la región de Kanto, actualizados con las mecánicas de la serie Escarlata y Púrpura.

A continuación, se presenta una tabla con las cinco cartas más destacadas de esta expansión, junto con su valor de mercado estimado en dólares estadounidenses y la probabilidad aproximada de obtenerlas en un sobre (pull rate):

Carta	Número | Valor de Mercado (USD) | Pull Rate Estimado
Charizard ex	199/165 |	$121.47 |	1 en 94 sobres
Blastoise ex	200/165	| $77.25 |	1 en 71 sobres
Venusaur ex	198/165	| $76.69 |	1 en 283 sobres
Zapdos ex	202/165	| $55.31 |	Datos no disponibles
Charmander	168/165	| $50.98 |	Datos no disponibles """

segundo_prompt = PromptTemplate(
    input_variables=["Cartas_Pokémon"],
    template=segunda_plantilla
)

# Creamos la segunda cadena
cadena_dos = LLMChain(llm=llm, prompt=segundo_prompt)

In [8]:
# Conectamos las cadenas

cadena_general = SimpleSequentialChain(chains=[cadena_uno, cadena_dos], verbose=True)
Expansión_Pokémon = "Crown Zenith"

respuesta_final = cadena_general.run(Expansión_Pokémon)



> Entering new SimpleSequentialChain chain...
"Arceus VSTAR 220/159, Giratina VSTAR 212/159, Mewtwo VSTAR 214/159, Pikachu 160/159, Charizard VSTAR 174/159"
La expansión "Crown Zenith" del Juego de Cartas Coleccionables Pokémon se lanzó el 20 de enero de 2023. Esta colección especial es parte de la serie Espada y Escudo y se destaca por incluir cartas con ilustraciones alternativas y versiones especiales de Pokémon populares. La colección consta de 230 cartas, incluyendo cartas secretas y variantes especiales.

A continuación, se presenta una tabla con las cinco cartas más destacadas de esta expansión, junto con su valor de mercado estimado en dólares estadounidenses y la probabilidad aproximada de obtenerlas en un sobre (pull rate):

| Carta               | Número  | Valor de Mercado (USD) | Pull Rate Estimado  |
|---------------------|---------|------------------------|---------------------|
| Arceus VSTAR        | 220/159 | $45.00                 | 1 en 100 sobres     |
| Giratina